In [1]:
from dataloader import Dataset
from utils import * 

data = Dataset("Stage_1_Data_ver._3/stage1/box")
data.load_data()

visibility = data.visibility

In [2]:
from scipy.spatial.transform import Rotation 
from scipy.sparse import lil_matrix
import time
from scipy.optimize import least_squares
def computeCurrentVisibility(visibility, reconstructed_ind, num_views, appended_ids):

    X_indices = (reconstructed_ind == 1)[:,0]
    current_visibility = visibility[:,appended_ids[:num_views]][X_indices]
    
    return X_indices, current_visibility

def computeCurrent2DPoints(X_indices, points_2d, appended_ids, current_visibility):
    
    current_2d_points = points_2d[:, appended_ids[:current_visibility.shape[1]]][X_indices]
    point_indices, camera_indices = current_visibility.nonzero()
    return current_2d_points[point_indices, camera_indices], point_indices, camera_indices
    
def projectFromCameras(X, cam_params, camera_indices, point_indices, K):

    projected = np.zeros((X.shape[0], cam_params.shape[0], 3))

    for camera in range(cam_params.shape[0]):

        R = Rotation.from_rotvec(cam_params[camera,:3]).as_matrix()
        C = cam_params[camera, 3:]
        P = computeProjectionMatrix(K, R, C)
        
        projected[:, camera] = projectPoint(P, X)

    return projected[point_indices, camera_indices][:,:2]

def objective(x0, num_views, n_points, camera_indices, point_indices, points_2d, K):
    
    camera_params = x0[:num_views * 6].reshape((num_views, 6))
    points_3d = x0[num_views * 6:].reshape((n_points, 3))
    points_proj = projectFromCameras(points_3d, camera_params,camera_indices,point_indices,K)
    error_vec = (points_proj - points_2d).ravel()

    return error_vec

def bundle_adjustment_sparsity(num_views, n_points, camera_indices, point_indices):
    m = camera_indices.size * 2
    n = num_views * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)
    for s in range(6):
        A[2 * i, camera_indices * 6 + s] = 1
        A[2 * i + 1, camera_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, num_views * 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, num_views * 6 + point_indices * 3 + s] = 1

    return A

def bundleAdjustment(X_reconstructed, reconstructed_ind, points_2d, visibility, camera_rotations, camera_translations, K, num_views, appended_ids):
    
    X_indices, current_visibility = computeCurrentVisibility(visibility, reconstructed_ind, num_views, appended_ids)
    X_3d = X_reconstructed[X_indices]
    n_points = X_3d.shape[0]
    X_2d,point_indices, camera_indices = computeCurrent2DPoints(X_indices, points_2d, appended_ids, current_visibility)

    cam_param_list = []
    for i in range(num_views):
        R, C = camera_rotations[i], camera_translations[i]
        rotvec = Rotation.from_matrix(R).as_rotvec()
        RC = [rotvec[0], rotvec[1], rotvec[2], C[0], C[1], C[2]]
        cam_param_list.append(RC)

    cam_param_list = np.array(cam_param_list).reshape(-1, 6)

    x0 = np.hstack((cam_param_list.ravel(), X_3d.ravel()))

    A = bundle_adjustment_sparsity(num_views,n_points, camera_indices, point_indices)
    res = least_squares(objective, x0, jac_sparsity=A, verbose=0, x_scale='jac', max_nfev=500, method='trf',
                        args=(num_views, n_points, camera_indices, point_indices,X_2d, K))
    
    optimized_params = res.x
    optimized_RC = optimized_params[:num_views * 6].reshape((num_views, 6))
    optimized_3d = optimized_params[num_views * 6:].reshape((n_points, 3))

    optimized_X = np.zeros_like(X_reconstructed)
    optimized_X[X_indices] = optimized_3d

    optimized_camera_rotations, optimized_camera_translation = [], []
    for i in range(len(optimized_RC)):
        R = Rotation.from_rotvec(optimized_RC[i, :3]).as_matrix()
        C = optimized_RC[i, 3:].reshape(3,1)
        optimized_camera_translation.append(C)
        optimized_camera_rotations.append(R)

    return optimized_X, optimized_camera_rotations, optimized_camera_rotations


In [3]:
images_to_add, appended_ids = data.computeSceneGraph()

In [4]:

indices    = np.logical_and(visibility[:,appended_ids[0]], visibility[:,appended_ids[1]])
pts1, pts2 = data.points[indices, appended_ids[0]], data.points[indices, appended_ids[1]]
E, best_criterion = estimate_essential_matrix(pts1, pts2, data.K)
rotations, translations = extract_cam_pose(E, data.K)
rot, trans = decomposeEssentialMat(E)
points_3d = []
for i in range(4):
    P1 = computeProjectionMatrix(data.K, np.identity(3), np.zeros((3,)))
    P2 = computeProjectionMatrix(data.K, rotations[i], translations[i])
    points_3d.append(linearTriangulation(pts1, pts2, P1, P2))

X,R,C = cheiralityCondition(points_3d, rotations, translations)

X_reconstructed   = np.zeros((data.points.shape[0], 3))
reconstructed_ind = np.zeros((data.points.shape[0], 1))

X_reconstructed[indices] = X
reconstructed_ind[(X_reconstructed[:,2] > 0)] = 1

camera_rotations    = []
camera_translations = []
camera_rotations.append(np.eye(3))
camera_rotations.append(R)
camera_translations.append(np.zeros((3,)))
camera_translations.append(C)


In [10]:
for next_image in range(2, 3):

    visibility_points = data.visibility[:, appended_ids[next_image]][...,None]
    new_points        = data.points[:,appended_ids[next_image]]
    indices           = np.logical_and(reconstructed_ind, visibility_points)[:,0]

    X_new       = X_reconstructed[indices]
    corr_points = new_points[indices]

    R_new, C_new = DLT_PnP(corr_points, X_new, data.K)
    camera_rotations.append(R_new)
    camera_translations.append(C_new[:,0])
    overlap = []
    for previous_images in range(next_image):
        
        indices = np.logical_and(np.logical_and(visibility[:,appended_ids[previous_images]][...,None], 
                                                visibility[:,appended_ids[next_image]][...,None]), 1 - reconstructed_ind)[:,0]
        overlap.append(np.logical_and(visibility[:,appended_ids[previous_images]][...,None],visibility[:,appended_ids[next_image]][...,None]))
        pts1, pts2 = data.points[indices, appended_ids[previous_images]], data.points[indices, appended_ids[next_image]]
        P1 = computeProjectionMatrix(data.K, camera_rotations[previous_images], camera_translations[previous_images])
        P2 = computeProjectionMatrix(data.K, R_new, C_new)
        points_3d = linearTriangulation(pts1, pts2, P1, P2)
        X_reconstructed[indices]   = points_3d
        reconstructed_ind[indices] = 1
    
    print(X_reconstructed.max(), X_reconstructed.min())
    X_reconstructed,camera_rotations,camera_translations = bundleAdjustment(X_reconstructed,reconstructed_ind,
                                                                            data.points,data.visibility,camera_rotations,
                                                                            camera_translations, data.K, next_image +1, appended_ids)
    
    
    

    

9791.881949024728 -13932.884166495127


In [5]:
X_reconstructed.min()

-7237.25493442442

In [6]:
X_reconstructed.max()

10146.392625340175

In [7]:
import  struct
def write_pointcloud(filename,xyz_points,rgb_points=None):

    """ creates a .pkl file of the point clouds generated
    """

    assert xyz_points.shape[1] == 3,'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8)*255
    assert xyz_points.shape == rgb_points.shape,'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    fid = open(filename,'wb')
    fid.write(bytes('ply\n', 'utf-8'))
    fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
    fid.write(bytes('element vertex %d\n'%xyz_points.shape[0], 'utf-8'))
    fid.write(bytes('property float x\n', 'utf-8'))
    fid.write(bytes('property float y\n', 'utf-8'))
    fid.write(bytes('property float z\n', 'utf-8'))
    fid.write(bytes('property uchar red\n', 'utf-8'))
    fid.write(bytes('property uchar green\n', 'utf-8'))
    fid.write(bytes('property uchar blue\n', 'utf-8'))
    fid.write(bytes('end_header\n', 'utf-8'))

    # Write 3D points to .ply file
    for i in range(xyz_points.shape[0]):
        fid.write(bytearray(struct.pack("fffccc",xyz_points[i,0],xyz_points[i,1],xyz_points[i,2],
                                        rgb_points[i,0].tostring(),rgb_points[i,1].tostring(),
                                        rgb_points[i,2].tostring())))
    fid.close()


In [8]:
X = X_reconstructed[(reconstructed_ind == 1)[...,0]]
c = data.colors[(reconstructed_ind == 1)[...,0]]

In [9]:
write_pointcloud("two_views.ply", X, c)

/tmp/ipykernel_21842/1081315607.py:28: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  rgb_points[i,0].tostring(),rgb_points[i,1].tostring(),
/tmp/ipykernel_21842/1081315607.py:29: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  rgb_points[i,2].tostring())))
